In [160]:
import pandas as pd
import numpy as np
import re
import unicodedata
import copy

In [161]:
def apply_rule(pattern,sen):
#     sen_list = re.split(r'([\,\.\?])',sen)
#     for part in sen_list:
#         if not part == ',' or part == '.' or part == '?':
#             sen = '.' + part + '.'
    sen = '.' + sen + '.'
    p1 = pattern.split('>>')[0]#字符串
    if re.findall(p1,sen):
        same_str = [x for x in (re.findall(p1,sen)[0])]
        answer=[]
        for item in p2:
            if '$@_' in item:
                nu = int(re.findall('[0-9]+',item)[0])
                curword = same_str[nu]
                if not curword.endswith(' '):
                     curword = curword + ' '
                if(curword.startswith(' ')):
                    curword = curword[1:]

                answer.append(curword)
            else:

                if not item.endswith(' '):
                     item = item + ' '
                if(item.startswith(' ')):
                    item = item[1:]
                answer.append(item)
#             answer_list.append(answer)
        return  (''.join(answer)[:-1])
#         else:
#             answer_list.append(part)
#     answer_list.append(''.join(answer_list)[:-1])
    else:
        return ""

In [162]:
df1 = pd.read_csv('quora.csv',encoding='gbk')
print(len(df1))
df1.head()

292007


,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What would happen if the Indian government sto...,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can Internet speed be increased by hacking...,How can I increase the speed of my internet co...
3,What keeps childern active and far from phone ...,Should I buy tiago ?
4,What should I do to be a great geologist ?,How can I be a good geologist ?


In [163]:
#rule 生成

In [229]:
rule=[]
for i in range(len(df1)):
    sentence1 = df1['question1'][i].lower().split()
    sentence1.pop()
    sentence2 = df1['question2'][i].lower().split()
    sentence2.pop()

    margin = 3
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break
    if flag:    
        r2_len = len(r2)
        sentence1_len = len(sentence1)
        id_dollar =0
        count = 0
        
        for word in sentence1:
            j = sentence1.index(word)
            try:
                id = sentence2.index(word)
                maxlen = 0
                xval = 0   

                while sentence1[j+xval] == sentence2[id+xval]:
                    maxlen = maxlen + 1 
                    xval = xval + 1
                    if j+xval >= sentence1_len or id+xval >=r2_len:
                        break
                if maxlen > 1:
                    while xval > 0:
                        del r2[id]
                        sentence2[id+maxlen-xval] = masked
                        del sentence1[j]
                        print('sentence1',sentence1)
                        print('r2',r2)
                        xval = xval - 1

                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                    print('r1',r1)
                    cur_word2 = '$@_{}'.format(id_dollar) 
                    id_dollar +=1
                    r2.insert(id,cur_word2)
                    print('r2',r2)
                elif maxlen == 1:

                    cur_word1 =r'(.+)'
                    r1+=cur_word1

                    cur_word2 = '$@_{}'.format(id_dollar) 
                    id_dollar +=1
                    r2[id] = cur_word2

                    sentence2[id] = masked
            except:
                cur_word1 = word           
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    

                r1+=cur_word1
                count = count + 1 
                print('r1',r1)
            if j<len(sentence1)-1:
                r1+='\s' 
                
        if count < margin or (r2_len-id_dollar-1)<margin:#r1、r2中至少有margin个单词以上
            flag = False
            
        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))
    if i > 10:
        break

sentence1 ['is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sentence1 ['the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sentence1 ['step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sentence1 ['by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sentence1 ['step', 'guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sentence1 ['guide', 'to', 'invest', 'in', 'share', 'market']
r2 ['guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india']
sent

In [226]:
print(len(rule))
rule[:10]

9


['[\\,\\.\\?](.+)\\swould\\shappen\\sif\\s(.+)\\sindian\\sgovernment\\sstole\\sthe\\s(.+)[\\,\\.\\?]>>$@_0 is $@_1 story of $@_2',
 '[\\,\\.\\?](.+)\\s(.+)\\sbe\\sincreased\\sby\\shacking\\sthrough\\sdns[\\,\\.\\?]>>$@_0 i increase the speed $@_1 my internet connection while using a vpn',
 '[\\,\\.\\?]what\\skeeps\\schildern\\sactive\\sand\\sfar\\sfrom\\sphone\\sand\\svideo\\sgames[\\,\\.\\?]>>should i buy tiago',
 '[\\,\\.\\?]what\\sshould\\s(.+)\\sdo\\sto\\s(.+)\\s(.+)geologist[\\,\\.\\?]>>how can $@_0 $@_1 good geologist',
 '[\\,\\.\\?](.+)\\sinstead\\s(.+)of\\s"and"[\\,\\.\\?]>>$@_0 シ instead of し',
 '[\\,\\.\\?]how\\sdo\\s(.+)\\s(.+)dcx3400\\sfor\\sfree\\sinternet[\\,\\.\\?]>>motorola (company): can $@_0 my charter motorolla dcx3400',
 '[\\,\\.\\?]what\\sare\\ssome\\s(.+)\\sthe\\sthings\\stechnicians\\scan\\stell\\sabout\\sthe\\sdurability\\sand\\sreliability\\sof\\slaptops\\sand\\sits\\scomponents[\\,\\.\\?]>>method to find separation $@_0 slits using fresnel biprism',
 '[\\,\\.\

In [118]:
sen = '.What should I do to be a great geologist.'.lower()
pattern = '[\,\.\?]what\sshould\s(.+)\sdo\sto\s(.+)\sgreat\sgeologist[\,\.\?]'
# re.findall(pattern,sen)
apply_rule(rule[2],sen)

'how much does it cost i setup a gaming channel on youtube'

In [174]:
for i in range(len(df1)):
    sentence1 = df1['question2'][i].lower().split()
    sentence1.pop()
    sentence2 = df1['question1'][i].lower().split()
    sentence2.pop()

    margin = 3
    flag = True
    masked = '%@%'
    
    r1 = ''   
    r2 = [x for x in sentence2]
    
    p = "\$@_\d+|\d+\$@_"#filter p2 that contains &@_number    
    for item in r2:
        if re.findall(p,item):
            flag = False
            break

    if flag:    
        r2_len = len(r2)

        id_dollar =0
        count = 0

        for word in sentence1:
            j = sentence1.index(word)
            try:                      
                id = sentence2.index(word)   
                maxlen = 1
                x = 1
                
                while(sentence1[j+x] == sentence2[id+x]):
                    maxlen = maxlen + 1 
                    x = x + 1
                    
                xval = maxlen
                
                if maxlen > 1:
                    while xval >= 1:
                        del r2[id]
                        sentence2[id+maxlen-xval] = masked
                        del sentence1[j]
                        
                        xval = xval - 1
                        
                    r2.insert(id,cur_word2)
                    
                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                else:
                    cur_word1 =r'(.+)'
                    r1+=cur_word1
                    
                    cur_word2 = '$@_{}'.format(id_dollar) 
                    id_dollar +=1
                    r2[id] = cur_word2
                    
                    sentence2[id] = masked
            except:
                cur_word1 = sentence1[j]            
                pattern = ".*[\\\|\*\[\]\{\}\|\(\)\+\.\?\$\^\/].*"
                if re.findall(pattern,cur_word1):            
                    flag=False
                    break    

                r1+=cur_word1
                count = count + 1 

            if j<len(sentence1)-1:
                r1+='\s'            

        if count < margin or (r2_len-id_dollar-1)<margin:
            flag = False

        if flag:
            r1 = '[\,\.\?]' + r1 + '[\,\.\?]'
            rule.append(''.join(r1) +'>>'+ ' '.join(r2))

In [175]:
# rule = list(set(rule))
print(len(rule))
rule[:5]

457864


['[\\,\\.\\?]what\\sis\\sthe\\sstep\\sby\\sstep\\sguide\\sto\\sinvest\\sin\\sshare\\smarket[\\,\\.\\?]>>what is the step by step guide to invest in share market in india',
 '[\\,\\.\\?](.+)\\s(.+)\\s(.+)\\sbe\\sincreased\\sby\\shacking\\sthrough\\sdns[\\,\\.\\?]>>$@_0 can i increase the $@_2 of my $@_1 connection while using a vpn',
 '[\\,\\.\\?]what\\skeeps\\schildern\\sactive\\sand\\sfar\\sfrom\\sphone\\sand\\svideo\\sgames[\\,\\.\\?]>>should i buy tiago',
 '[\\,\\.\\?]what\\sshould\\s(.+)\\sdo\\sto\\s(.+)\\sgreat\\sgeologist[\\,\\.\\?]>>how can $@_0 $@_1 a good geologist',
 '[\\,\\.\\?]how\\sdo\\s(.+)\\s(.+)\\sdcx3400\\sfor\\sfree\\sinternet[\\,\\.\\?]>>$@_1 (company): can $@_0 hack my charter motorolla dcx3400']

In [176]:
#测试

In [ ]:
sen = 'I want to learn coding how should I start it'.lower()
print("rule[0]",rule[0])
print(apply_rule(rule[0],sen))

In [ ]:
#rule 排序

In [177]:
dic = {}
for r in rule:
    if r in dic:
        dic[r]+=1
    else:
        dic[r]=1
sort_rule1 = sorted(dic.items(), key=lambda e:e[1], reverse=True)

In [178]:
rules=[]
count=[]
for i in range(len(sort_rule1)):
    rules.append(sort_rule1[i][0])
    count.append(sort_rule1[i][1])

In [180]:
print(len(rules))
print(rules[:5])
print(count[:5])

456347
['[\\,\\.\\?](.+)\\sand\\swhat\\sshould\\speople\\sknow\\sabout\\sthem[\\,\\.\\?]>>$@_0 india and what should people know about them', "[\\,\\.\\?](.+)\\s'\\sused\\sin\\sa\\ssentence[\\,\\.\\?]>>$@_0 'adduce' used in a sentence", "[\\,\\.\\?](.+)\\s'\\sused\\sin\\sa\\ssentence[\\,\\.\\?]>>$@_0 'mischievous' used in a sentence", "[\\,\\.\\?](.+)\\s'\\sused\\sin\\sa\\ssentence[\\,\\.\\?]>>$@_0 'gape' used in a sentence", '[\\,\\.\\?](.+)\\snot\\sbe\\sallowed\\sto\\svote[\\,\\.\\?]>>$@_0 96 not be allowed to vote']
[17, 13, 11, 9, 9]


In [136]:
ruless = []
for rule in rules:
    count0 = 0
    
    p1 = rule.split('>>')[1].split()
    p2 = rule.split('>>')[1].split()
    
    pattern1 = "\$\d+"
    pattern2 = "\d+\$"
    
    flag_filter = False
    
    for item1 in p1:
        list_test1 = re.findall(pattern1,item1)
        list_test2 = re.findall(pattern2,item1)
        
        if list_test1 or list_test2:
            flag_filter = True

    for item2 in p2:
        list_test1 = re.findall(pattern1,item2)
        list_test2 = re.findall(pattern2,item2)
        
        if list_test1 or list_test2:
                flag_filter = True
       
        if '$' == item2:
            flag_filter = True
               
    
        if '$@_' in item2:
            count0 = count0 + 1            
    if flag_filter:
        continue
        
    if count0 > 0:
        ruless.append(rule)
        
print(len(ruless))
print(ruless[:5])

387709
['[\\,\\.\\?](.+)\\sthere\\sany\\snovels\\sthat\\suse\\sprofanity[\\,\\.\\?]>>what $@_0 the best and profitable items to export to dubai from india', '[\\,\\.\\?]who\\s(.+)\\sbe\\sthe\\snext\\spresident\\sof\\susa:\\s(.+)\\sclinton\\sor\\sdonald\\strump[\\,\\.\\?]>>$@_0 donald trump beat $@_1 and become our next president', '[\\,\\.\\?](.+)\\sdo\\speople\\searn\\smoney\\sfrom\\syoutube[\\,\\.\\?]>>$@_0 winning money from youtube', '[\\,\\.\\?](.+)\\sis\\s(.+)\\ssuch\\sa\\sdivisive\\sartist[\\,\\.\\?]>>$@_0 are the $@_1 warhol soup can paintings considered great art', '[\\,\\.\\?]what\\s(.+)\\ssome\\s(.+)\\sall\\stime[\\,\\.\\?]>>which $@_0 the $@_0 in the world']


In [137]:
with open('rule_filter_test.txt','w') as f:
    for r in ruless:
        try:
            f.write(r)
            f.write('\n')
        except:
            r = r.encode("gb18030")
            r = str(r) 

In [ ]:
with open('rule_filter_test.txt','r') as f:
    ruless = f.read().split('\n')
    ruless.pop()
print(ruless[:10])

In [119]:
sen_fail = []
count =[]
for i in range(len(df1)):
    count.append(0)
    sen=df1['question1'][i].lower()
    sen = sen.replace(' ?','')
    flag1_ = True
    for j in range(len(ruless)):
        answer = apply_rule(sen,ruless[j])
        if answer:
            flag1_ = False
            print('\n','sen',sen,'\n','rule',ruless[j],'\n','answer',answer,'\n')
            count[i] = count[i] + 1
    if flag1_:
        sen_fail.append(df1['question1'][i])
    if i > 20:
        break

In [27]:
print(sen_fail)
for item in count:
    print(item)

['What is the step by step guide to invest in share market ?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back ?', 'How can Internet speed be increased by hacking through DNS ?', 'What keeps childern active and far from phone and video games ?', 'What should I do to be a great geologist ?', 'When do you use "&" instead of "and" ?', 'How do I hack Motorola DCX3400 for free internet ?', 'What are some of the things technicians can tell about the durability and reliability of Laptops and its components ?', 'How can I see all my Youtube comments ?', 'How can you make physics easy to learn ?', 'What was your first sexual experience ?', 'How will a Trump presidency affect the students presently in US or planning to study in US ?', 'What does manipulation means ?', 'How do guys feel after rejecting a girl ?', 'Why do people ask Quora questions which can be answered easily by Google ?', 'Which is the best digital marketing institute in Pune ?', 'Why are

In [ ]:
word = 'dsafa'
sen = 'dsafs dsafa dsaf'
sen = sen.split()
id = sen.index(word)
del sen[id]
del sen[id]
print(sen)
sen.insert(1,'fdfds')
print(sen)